# Pricing y Valorización de Swaps

Se importan las librerías necesarias.

In [1]:
from finrisk import QC_Financial_3 as Qcf
import pandas as pd

Objeto auxiliar para formatear el output de los `pandas.DataFrame`.

In [2]:
format_dict = {'nominal': '{0:,.2f}', 'amort': '{0:,.2f}', 'interes': '{0:,.2f}', 'flujo': '{0:,.2f}',
               'icp_inicial': '{0:,.2f}', 'icp_final': '{0:,.2f}',
               'valor_tasa': '{0:,.4%}', 'spread': '{0:,.4%}', 'gearing': '{0:,.2f}',
               'amort_moneda_pago': '{0:,.2f}', 'interes_moneda_pago': '{0:,.2f}', 'valor_indice_fx': '{0:,.2f}'}

## Ejemplo Usando QC_Financial

### Fecha de valorización

In [3]:
fecha_hoy = Qcf.QCDate(6, 9, 2019)

### Curva Cero Cupón

Es la curva construida con las tasas de cierre del viernes 2019-09-06. Las tasas de esta curva están en convención **Lineal Act/360**.

In [41]:
curva = pd.read_excel("20190906_curva_camara_clp.xlsx")
curva.style.format({"tasa":"{0:,.8%}"})

,fecha,plazo,tasa
0,2019-09-06 00:00:00,3,2.00000000%
1,2019-09-06 00:00:00,4,2.00008333%
2,2019-09-06 00:00:00,95,1.92857119%
3,2019-09-06 00:00:00,186,1.77533292%
4,2019-09-06 00:00:00,278,1.69483180%
5,2019-09-06 00:00:00,370,1.65909445%
6,2019-09-06 00:00:00,551,1.62808143%
7,2019-09-06 00:00:00,735,1.64202926%
8,2019-09-06 00:00:00,1100,1.76534701%
9,2019-09-06 00:00:00,1465,1.92308825%


Se define un objeto curva de `Qcf`.

In [5]:
lvec1 = Qcf.long_vec()
vec1 = Qcf.double_vec()
for index, row in curva.iterrows():
    lvec1.append(int(row['plazo']))
    vec1.append(row['tasa'])
    
zcc1 = Qcf.QCCurve(lvec1, vec1)
lin1 = Qcf.QCLinearInterpolator(zcc1)
tasa = Qcf.QCInterestRate(.01, Qcf.QCAct360(), Qcf.QCLinearWf())
zz1 = Qcf.ZeroCouponCurve(lin1, tasa)

In [6]:
print("Check: {0:,.8%}".format(zz1.get_rate_at(100)))

Check: 1.92015150%


In [7]:
zz1.get_discount_factor_at(100)

0.9946945438277722

In [8]:
print("Check: {0:,.8%}".format((1.92857119 + (1.77533292 - 1.92857119) / (186 - 95) * (100 - 95)) / 100))

Check: 1.92015150%


### Pata Fija

Daremos de alta un swap a 5Y. Primero se construye la pata fija. La tasa fija corresponde al mid-point de cierre.

In [77]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(10, 9, 2019)
fecha_final = Qcf.QCDate(10, 9, 2024) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad = Qcf.Tenor('6M')
periodo_irregular = Qcf.StubPeriod.SHORTFRONT
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100000000.0
amort_es_flujo = True
valor_tasa_fija = .0196
tasa_cupon = Qcf.QCInterestRate(valor_tasa_fija, Qcf.QCAct360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLP()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(rp,
                                                            fecha_inicio,
                                                            fecha_final,
                                                            bus_adj_rule,
                                                            periodicidad,
                                                            periodo_irregular,
                                                            calendario,
                                                            lag_pago,
                                                            nominal,
                                                            amort_es_flujo,
                                                            tasa_cupon,
                                                            moneda,
                                                            es_bono)

In [78]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(Qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df.style.format(format_dict)

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-09-10,2020-03-10,2020-03-10,"100,000,000.00",0.00,"990,888.89",True,"990,888.89",CLP,1.9600%,LinAct360
1,2020-03-10,2020-09-10,2020-09-10,"100,000,000.00",0.00,"1,001,777.78",True,"1,001,777.78",CLP,1.9600%,LinAct360
2,2020-09-10,2021-03-10,2021-03-10,"100,000,000.00",0.00,"985,444.44",True,"985,444.44",CLP,1.9600%,LinAct360
3,2021-03-10,2021-09-10,2021-09-10,"100,000,000.00",0.00,"1,001,777.78",True,"1,001,777.78",CLP,1.9600%,LinAct360
4,2021-09-10,2022-03-10,2022-03-10,"100,000,000.00",0.00,"985,444.44",True,"985,444.44",CLP,1.9600%,LinAct360
5,2022-03-10,2022-09-12,2022-09-12,"100,000,000.00",0.00,"1,012,666.67",True,"1,012,666.67",CLP,1.9600%,LinAct360
6,2022-09-12,2023-03-10,2023-03-10,"100,000,000.00",0.00,"974,555.56",True,"974,555.56",CLP,1.9600%,LinAct360
7,2023-03-10,2023-09-11,2023-09-11,"100,000,000.00",0.00,"1,007,222.22",True,"1,007,222.22",CLP,1.9600%,LinAct360
8,2023-09-11,2024-03-11,2024-03-11,"100,000,000.00",0.00,"990,888.89",True,"990,888.89",CLP,1.9600%,LinAct360
9,2024-03-11,2024-09-10,2024-09-10,"100,000,000.00","100,000,000.00","996,333.33",True,"100,996,333.33",CLP,1.9600%,LinAct360


#### Valor Presente Pata Fija

Se calcula el valor presente.

In [11]:
pv = Qcf.PresentValue()

In [71]:
vp_fija = pv.pv(fecha_hoy, fixed_rate_leg, zz1)
print("Valor presente de la pata fija es: {0:,.4f}".format(vp_fija))

Valor presente de la pata fija es: 99,977,781.7894


¿No debiera valer justo 100,000,000 CLP? Es el input de cierre valorizado a la curva de cierre.

Hay que recordar que la pata fija vale el nocional (si se considera el nocional como flujo) si se decuenta hasta fecha de inicio. En el cálculo anterior se descontó hasta fecha de hoy.

Se calcula el factor de descuento hasta fecha de inicio.

In [13]:
df2d = zz1.get_discount_factor_at(4) # 4 días porque es de viernes a martes.

In [14]:
r = zz1.get_rate_at(4)

In [15]:
vp_fija * (1 + r * 4/360)

100000000.00000247

Se capitaliza el valor a fecha de hoy hasta fecha de inicio.

In [16]:
print("¡Ahora sí! El valor presente a fecha de inicio es: {0:,.4f}".format(vp_fija / df2d))

¡Ahora sí! El valor presente a fecha de inicio es: 100,000,000.0000


### Pata Flotante (Cámara)

Se construye la pata flotante.

In [17]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.PAY
fecha_inicio = Qcf.QCDate(10, 9, 2019)
fecha_final = Qcf.QCDate(10, 9, 2024) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = Qcf.Tenor('6M')
periodo_irregular_pago = Qcf.StubPeriod.NO
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100000000.0
amort_es_flujo = True 
spread = .0
gearing = 1.0

icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp_leg(rp, fecha_inicio, fecha_final, bus_adj_rule,
                                                      periodicidad_pago,
                                                      periodo_irregular_pago, calendario, lag_pago,
                                                      nominal, amort_es_flujo, spread, gearing)

In [18]:
## Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp_leg.size()):
    tabla.append(Qcf.show(icp_clp_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df_icp = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df_icp.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-09-10,2020-03-10,2020-03-10,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
1,2020-03-10,2020-09-10,2020-09-10,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
2,2020-09-10,2021-03-10,2021-03-10,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
3,2021-03-10,2021-09-10,2021-09-10,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
4,2021-09-10,2022-03-10,2022-03-10,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
5,2022-03-10,2022-09-12,2022-09-12,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
6,2022-09-12,2023-03-10,2023-03-10,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
7,2023-03-10,2023-09-11,2023-09-11,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
8,2023-09-11,2024-03-11,2024-03-11,"-100,000,000.00",0.00,True,0.00,CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360
9,2024-03-11,2024-09-10,2024-09-10,"-100,000,000.00","-100,000,000.00",True,"-100,000,000.00",CLP,"10,000.00","10,000.00",0.0000%,-0.00,0.0000%,1.00,LinAct360


**Notar que los valores de las tasas de devengo no están fijados.**

Vamos a calcular tasas forward.

#### Cálculo de Tasas Forward

In [19]:
fwd_rates = Qcf.ForwardRates()

Para valorizar, es conveniente definir que la TNA se calcula a más de 2 decimales. Con la convención fijada en los contratos se pierde precisión.

In [20]:
for i in range(0, icp_clp_leg.size()):
    icp_clp_leg.get_cashflow_at(i).set_tna_decimal_places(18) # <---- Aquí

Se calculan y establecen las tasas forward.

In [21]:
fwd_rates.set_rates_icp_clp_leg(fecha_hoy, 18719.83, icp_clp_leg, zz1)

Miramos nuevamente los flujos.

In [22]:
## Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp_leg.size()):
    tabla.append(Qcf.show(icp_clp_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df_icp = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df_icp.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-09-10,2020-03-10,2020-03-10,"-100,000,000.00",0.00,True,"-894,833.33",CLP,"18,723.99","18,891.54",1.7700%,"-894,833.33",0.0000%,1.00,LinAct360
1,2020-03-10,2020-09-10,2020-09-10,"-100,000,000.00",0.00,True,"-780,763.47",CLP,"18,891.54","19,039.04",1.5276%,"-780,763.47",0.0000%,1.00,LinAct360
2,2020-09-10,2021-03-10,2021-03-10,"-100,000,000.00",0.00,True,"-773,499.12",CLP,"19,039.04","19,186.30",1.5385%,"-773,499.12",0.0000%,1.00,LinAct360
3,2021-03-10,2021-09-10,2021-09-10,"-100,000,000.00",0.00,True,"-839,683.51",CLP,"19,186.30","19,347.41",1.6429%,"-839,683.51",0.0000%,1.00,LinAct360
4,2021-09-10,2022-03-10,2022-03-10,"-100,000,000.00",0.00,True,"-949,347.27",CLP,"19,347.41","19,531.08",1.8882%,"-949,347.27",0.0000%,1.00,LinAct360
5,2022-03-10,2022-09-12,2022-09-12,"-100,000,000.00",0.00,True,"-1,028,353.60",CLP,"19,531.08","19,731.93",1.9904%,"-1,028,353.60",0.0000%,1.00,LinAct360
6,2022-09-12,2023-03-10,2023-03-10,"-100,000,000.00",0.00,True,"-1,094,301.06",CLP,"19,731.93","19,947.86",2.2008%,"-1,094,301.06",0.0000%,1.00,LinAct360
7,2023-03-10,2023-09-11,2023-09-11,"-100,000,000.00",0.00,True,"-1,194,379.71",CLP,"19,947.86","20,186.11",2.3242%,"-1,194,379.71",0.0000%,1.00,LinAct360
8,2023-09-11,2024-03-11,2024-03-11,"-100,000,000.00",0.00,True,"-1,190,762.18",CLP,"20,186.11","20,426.48",2.3554%,"-1,190,762.18",0.0000%,1.00,LinAct360
9,2024-03-11,2024-09-10,2024-09-10,"-100,000,000.00","-100,000,000.00",True,"-101,246,813.30",CLP,"20,426.48","20,681.16",2.4527%,"-1,246,813.30",0.0000%,1.00,LinAct360


#### Valor Presente

El valor presente es el esperado:

In [23]:
print("Valor presente pata flotante: {0:,.4f}".format(round(pv.pv(fecha_hoy, icp_clp_leg, zz1) / df2d, 6)))

Valor presente pata flotante: -100,000,000.0000


## Ejercicio 1

Replique (o compruebe) los cálculos de la librería. Puede usar Excel o Python o calculadora o papel.

- Chequee el valor presente de la pata fija
- Chequee el valor presente de la pata ICP
- Si quiere replique la construcción de las patas (fechas de inicio, final y de pago) y cálculo de intereses

*Reverse engineer the library*

### Solución

**Tip**: Diferencia en días entre 2 fechas:

In [24]:
fecha = Qcf.QCDate()
print("Fecha más antigua: {}".format(fecha.description(False)))
print("Fecha más reciente: {}".format(fecha_hoy.description(False)))
print("Número de días entre ambas fechas: {0:,.0f}".format(fecha.day_diff(fecha_hoy)))

Fecha más antigua: 1969-01-12
Fecha más reciente: 2019-09-06
Número de días entre ambas fechas: 18,499


Calcular:

- el número de días entre la fecha de inicio y fecha final de cada cupón de la pata fija,
- el número de días desde fecha de hoy hasta la fecha de pago de cada cupón y
- el factor de descuento apropiado para ese plazo.

In [63]:
dias = []
for index, row in df.iterrows():
    f1 = row['fecha_ini']
    qc_f1 = Qcf.build_qcdate_from_string(f1)
    
    f2 = row['fecha_fin']
    qc_f2 = Qcf.build_qcdate_from_string(f2)
    
    f3 = row['fecha_pago']
    qc_f3 = Qcf.build_qcdate_from_string(f3)
    
    p = fecha_hoy.day_diff(qc_f3)
    dias.append((qc_f1.day_diff(qc_f2), p, zz1.get_discount_factor_at(p)))

for d in dias:
    print(d)

(182, 186, 0.9909108175949431)
(184, 370, 0.9832340850516539)
(181, 551, 0.97568715349)
(184, 735, 0.9675626891427448)
(181, 916, 0.958463541674464)
(186, 1102, 0.948707474239714)
(179, 1281, 0.938438135789829)
(185, 1466, 0.9273619132965925)
(182, 1648, 0.916449183010472)
(183, 1831, 0.9051634843011306)


Calcular intereses pata fija:

In [66]:
intereses = []
for index, row in df.iterrows():
    intereses.append(nominal * valor_tasa_fija * dias[index][0] / 360)

index = 0
for i in intereses:
    print("Interés cupón {0:}: {1:,.2f}".format(index, i))
    index += 1

Interés cupón 0: 990,888.89
Interés cupón 1: 1,001,777.78
Interés cupón 2: 985,444.44
Interés cupón 3: 1,001,777.78
Interés cupón 4: 985,444.44
Interés cupón 5: 1,012,666.67
Interés cupón 6: 974,555.56
Interés cupón 7: 1,007,222.22
Interés cupón 8: 990,888.89
Interés cupón 9: 996,333.33


Calcular valor presente pata fija:

In [74]:
check_vp = 0
for i in range(0, 10):
    check_vp += intereses[i] * dias[i][2]
    if i == 9:
        check_vp += nominal * dias[i][2]
print("check_vp: {0:,.6f}".format(check_vp))
print("Diferencia: {0:,.6f}".format(vp_fija - check_vp))

check_vp: 99,977,781.789440
Diferencia: -0.000000


Cálculo tasa forward:

$$
FWD\left(d_1,d_2\right)=\left(\frac{df(d_1)}{df(d_2)}-1\right)\cdot\frac{360}{d_2-d_1}
$$

Los días $d_1$ y $d_2$ se calculan desde fecha de hoy hasta fecha de inicio y desde fecha de hoy hasta fecha final, respectivamente para cada cupón.

Se calculan los números de días necesarios para el cálculo de tasas forward:

In [79]:
dias_para_forward = []
for index, row in df.iterrows():
    f1 = row['fecha_ini']
    qc_f1 = Qcf.build_qcdate_from_string(f1)
    
    f2 = row['fecha_fin']
    qc_f2 = Qcf.build_qcdate_from_string(f2)
    
    p1 = fecha_hoy.day_diff(qc_f1)
    p2 = fecha_hoy.day_diff(qc_f2)
    dias_para_forward.append((p1, p2))

for d in dias_para_forward:
    print(d)

(4, 186)
(186, 370)
(370, 551)
(551, 735)
(735, 916)
(916, 1102)
(1102, 1281)
(1281, 1466)
(1466, 1648)
(1648, 1831)


Se calculan las tasas forward:

In [82]:
tasas_forward = []
for d in dias_para_forward:
    df1 = zz1.get_discount_factor_at(d[0])
    df2 = zz1.get_discount_factor_at(d[1])
    tasas_forward.append((df1 / df2 - 1) * 360 / (d[1] - d[0]))

i = 0
for r in tasas_forward:
    print("Tasa forward cupón {0:}: {1:,.4%}".format(i, r))
    i += 1

Tasa forward cupón 0: 1.7700%
Tasa forward cupón 1: 1.5276%
Tasa forward cupón 2: 1.5385%
Tasa forward cupón 3: 1.6429%
Tasa forward cupón 4: 1.8882%
Tasa forward cupón 5: 1.9904%
Tasa forward cupón 6: 2.2008%
Tasa forward cupón 7: 2.3242%
Tasa forward cupón 8: 2.3554%
Tasa forward cupón 9: 2.4527%


En el mismo loop, se calculan los intereses y el valor presente de la pata flotante (ICP).

In [84]:
vp_icp = 0
for i in range(0, 10):
    df = dias[i][2]
    vp_icp += nominal * tasas_forward[i] * dias[i][0] / 360 * df
    if i == 9:
        vp_icp += nominal * df

print("check_icp: {0:,.4f}".format(vp_icp))

check_icp: 99,977,781.7894


## Ejercicio 2

Encuentre la tasa fija que hace que el valor presente del swap con las siguientes patas sea 0.

### Pata Fija

In [25]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(10, 9, 2019)
fecha_final = Qcf.QCDate(10, 9, 2029) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad = Qcf.Tenor('1Y')
periodo_irregular = Qcf.StubPeriod.SHORTFRONT
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100000000.0
amort_es_flujo = True

valor_tasa_fija = .02 # <---- Esta es la tasa que hay que calcular.

tasa_cupon = Qcf.QCInterestRate(valor_tasa_fija, Qcf.QCAct360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLP()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(rp,
                                                            fecha_inicio,
                                                            fecha_final,
                                                            bus_adj_rule,
                                                            periodicidad,
                                                            periodo_irregular,
                                                            calendario,
                                                            lag_pago,
                                                            nominal,
                                                            amort_es_flujo,
                                                            tasa_cupon,
                                                            moneda,
                                                            es_bono)

# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(Qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df.style.format(format_dict)

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-09-10,2020-09-10,2020-09-10,"100,000,000.00",0.00,"2,033,333.33",True,"2,033,333.33",CLP,2.0000%,LinAct360
1,2020-09-10,2021-09-10,2021-09-10,"100,000,000.00",0.00,"2,027,777.78",True,"2,027,777.78",CLP,2.0000%,LinAct360
2,2021-09-10,2022-09-12,2022-09-12,"100,000,000.00",0.00,"2,038,888.89",True,"2,038,888.89",CLP,2.0000%,LinAct360
3,2022-09-12,2023-09-11,2023-09-11,"100,000,000.00",0.00,"2,022,222.22",True,"2,022,222.22",CLP,2.0000%,LinAct360
4,2023-09-11,2024-09-10,2024-09-10,"100,000,000.00",0.00,"2,027,777.78",True,"2,027,777.78",CLP,2.0000%,LinAct360
5,2024-09-10,2025-09-10,2025-09-10,"100,000,000.00",0.00,"2,027,777.78",True,"2,027,777.78",CLP,2.0000%,LinAct360
6,2025-09-10,2026-09-10,2026-09-10,"100,000,000.00",0.00,"2,027,777.78",True,"2,027,777.78",CLP,2.0000%,LinAct360
7,2026-09-10,2027-09-10,2027-09-10,"100,000,000.00",0.00,"2,027,777.78",True,"2,027,777.78",CLP,2.0000%,LinAct360
8,2027-09-10,2028-09-11,2028-09-11,"100,000,000.00",0.00,"2,038,888.89",True,"2,038,888.89",CLP,2.0000%,LinAct360
9,2028-09-11,2029-09-10,2029-09-10,"100,000,000.00","100,000,000.00","2,022,222.22",True,"102,022,222.22",CLP,2.0000%,LinAct360


### Pata Flotante

In [38]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.PAY
fecha_inicio = Qcf.QCDate(10, 9, 2019)
fecha_final = Qcf.QCDate(10, 9, 2029) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = Qcf.Tenor('6M')
periodo_irregular_pago = Qcf.StubPeriod.NO
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100000000.0
amort_es_flujo = True 
spread = .02
gearing = 1

icp_clp_leg = Qcf.LegFactory.build_bullet_icp_clp_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                      periodo_irregular_pago, calendario, lag_pago,
                                                      nominal, amort_es_flujo, spread, gearing)

## Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, icp_clp_leg.size()):
    tabla.append(Qcf.show(icp_clp_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_inicial', 'fecha__final', 'fecha__pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df_icp = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df_icp.style.format(format_dict)

,fecha_inicial,fecha__final,fecha__pago,nominal,amort,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-09-10,2020-03-10,2020-03-10,"-100,000,000.00",0.00,True,"-1,011,111.11",CLP,"10,000.00","10,000.00",0.0000%,"-1,011,111.11",2.0000%,1.00,LinAct360
1,2020-03-10,2020-09-10,2020-09-10,"-100,000,000.00",0.00,True,"-1,022,222.22",CLP,"10,000.00","10,000.00",0.0000%,"-1,022,222.22",2.0000%,1.00,LinAct360
2,2020-09-10,2021-03-10,2021-03-10,"-100,000,000.00",0.00,True,"-1,005,555.56",CLP,"10,000.00","10,000.00",0.0000%,"-1,005,555.56",2.0000%,1.00,LinAct360
3,2021-03-10,2021-09-10,2021-09-10,"-100,000,000.00",0.00,True,"-1,022,222.22",CLP,"10,000.00","10,000.00",0.0000%,"-1,022,222.22",2.0000%,1.00,LinAct360
4,2021-09-10,2022-03-10,2022-03-10,"-100,000,000.00",0.00,True,"-1,005,555.56",CLP,"10,000.00","10,000.00",0.0000%,"-1,005,555.56",2.0000%,1.00,LinAct360
5,2022-03-10,2022-09-12,2022-09-12,"-100,000,000.00",0.00,True,"-1,033,333.33",CLP,"10,000.00","10,000.00",0.0000%,"-1,033,333.33",2.0000%,1.00,LinAct360
6,2022-09-12,2023-03-10,2023-03-10,"-100,000,000.00",0.00,True,"-994,444.44",CLP,"10,000.00","10,000.00",0.0000%,"-994,444.44",2.0000%,1.00,LinAct360
7,2023-03-10,2023-09-11,2023-09-11,"-100,000,000.00",0.00,True,"-1,027,777.78",CLP,"10,000.00","10,000.00",0.0000%,"-1,027,777.78",2.0000%,1.00,LinAct360
8,2023-09-11,2024-03-11,2024-03-11,"-100,000,000.00",0.00,True,"-1,011,111.11",CLP,"10,000.00","10,000.00",0.0000%,"-1,011,111.11",2.0000%,1.00,LinAct360
9,2024-03-11,2024-09-10,2024-09-10,"-100,000,000.00",0.00,True,"-1,016,666.67",CLP,"10,000.00","10,000.00",0.0000%,"-1,016,666.67",2.0000%,1.00,LinAct360
